In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#Загружаем исходные датасеты
train = pd.read_csv('drive/My Drive/repos/ReChorus/data/amazon_electronics/train.csv', sep='\t').loc[:, ['user_id', 'item_id', 'time']]
test = pd.read_csv('drive/My Drive/repos/ReChorus/data/amazon_electronics/test.csv', sep='\t').loc[:, ['user_id', 'item_id', 'time', 'neg_items']]
dev = pd.read_csv('drive/My Drive/repos/ReChorus/data/amazon_electronics/dev.csv', sep='\t').loc[:, ['user_id', 'item_id', 'time', 'neg_items']]

In [8]:
#Вычисляем временные интервалы между интеракциями одного пользователя
test.index = np.arange(train.shape[0], train.shape[0]+test.shape[0])
dev.index = np.arange(train.shape[0]+test.shape[0], train.shape[0]+test.shape[0]+dev.shape[0])

all_data = pd.concat([train, test, dev]).sort_values('time')
all_data['diff'] = all_data.groupby('user_id')['time'].transform(pd.Series.diff).fillna(0)

train['diff'] = all_data.loc[train.index, 'diff']
test['diff'] = all_data.loc[test.index, 'diff']
dev['diff'] = all_data.loc[dev.index, 'diff']

In [9]:
#Нормируем непрерывные признаки
train['time1'] = np.vectorize(lambda x: np.sum(train['time'] <= x) / train.shape[0])(train['time'])
test['time1'] = np.vectorize(lambda x: np.sum(train['time'] <= x) / train.shape[0])(test['time'])
dev['time1'] = np.vectorize(lambda x: np.sum(train['time'] <= x) / train.shape[0])(dev['time'])

train['diff1'] = np.vectorize(lambda x: np.sum(train['diff'] <= x) / train.shape[0])(train['diff'])
test['diff1'] = np.vectorize(lambda x: np.sum(train['diff'] <= x) / train.shape[0])(test['diff'])
dev['diff1'] = np.vectorize(lambda x: np.sum(train['diff'] <= x) / train.shape[0])(dev['diff'])

In [12]:
train.to_csv('drive/My Drive/repos/ReChorus/data/amazon_electronics1/train.csv', sep='\t', index=False)
test.to_csv('drive/My Drive/repos/ReChorus/data/amazon_electronics1/test.csv', sep='\t', index=False)
dev.to_csv('drive/My Drive/repos/ReChorus/data/amazon_electronics1/dev.csv', sep='\t', index=False)